In [6]:
import os, shutil
original_dataset = 'chest_xray/'
base_dir = 'chest_xray_small/'
os.mkdir(base_dir)
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

val_dir = os.path.join(base_dir, 'val')
os.mkdir(val_dir)

train_normal_dir = os.path.join(train_dir, 'normal')
os.mkdir(train_normal_dir)

train_pneumonia_dir = os.path.join(train_dir, 'pneumonia')
os.mkdir(train_pneumonia_dir)

test_normal_dir = os.path.join(test_dir, 'normal')
os.mkdir(test_normal_dir)

test_pneumonia_dir = os.path.join(test_dir, 'pneumonia')
os.mkdir(test_pneumonia_dir)

val_normal_dir = os.path.join(val_dir, 'normal')
os.mkdir(val_normal_dir)

val_pneumonia_dir = os.path.join(val_dir, 'pneumonia')
os.mkdir(val_pneumonia_dir)

In [13]:
print('Total training normal images:', len(os.listdir('chest_xray/train/NORMAL')))
print('Total training pneumonia images:', len(os.listdir('chest_xray/train/PNEUMONIA')))

print('Total test normal images:', len(os.listdir('chest_xray/test/NORMAL')))
print('Total test pneumonia images:', len(os.listdir('chest_xray/test/PNEUMONIA')))

print('Total validation normal images:', len(os.listdir('chest_xray/val/NORMAL')))
print('Total validation pneumonia images:', len(os.listdir('chest_xray/val/PNEUMONIA')))

Total training normal images: 1342
Total training pneumonia images: 3876
Total test normal images: 234
Total test pneumonia images: 390
Total validation normal images: 9
Total validation pneumonia images: 9


In [32]:
fnames = ['NORMAL ({}).jpeg'.format(i) for i in range(1, 1301)]
for fname in fnames:
    src = os.path.join(original_dataset + 'train/NORMAL/', fname)
    dst = os.path.join(train_normal_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['PNEUMONIA ({}).jpeg'.format(i) for i in range(1, 1301)]
for fname in fnames:
    src = os.path.join(original_dataset + 'train/PNEUMONIA/', fname)
    dst = os.path.join(train_pneumonia_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['NORMAL ({}).jpeg'.format(i) for i in range(1, 231)]
for fname in fnames:
    src = os.path.join(original_dataset + 'test/NORMAL/', fname)
    dst = os.path.join(test_normal_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['PNEUMONIA ({}).jpeg'.format(i) for i in range(1, 231)]
for fname in fnames:
    src = os.path.join(original_dataset + 'test/PNEUMONIA/', fname)
    dst = os.path.join(test_pneumonia_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['NORMAL ({}).jpeg'.format(i) for i in range(1, 9)]
for fname in fnames:
    src = os.path.join(original_dataset + 'val/NORMAL/', fname)
    dst = os.path.join(val_normal_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['PNEUMONIA ({}).jpeg'.format(i) for i in range(1, 9)]
for fname in fnames:
    src = os.path.join(original_dataset + 'val/PNEUMONIA/', fname)
    dst = os.path.join(val_pneumonia_dir, fname)
    shutil.copyfile(src, dst)

In [339]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization

model = models.Sequential()
model.add(layers.Conv2D(16, (3,3),input_shape=(150,150,3), use_bias = False))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization(scale = False, center=True))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(32, (3,3), use_bias = False))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization(scale = False, center=True))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64, (3,3), use_bias = False))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization(scale = False, center=True))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(128, (3,3),use_bias = False ))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization(scale = False, center=True))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.4))

model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [340]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 148, 148, 16)      432       
_________________________________________________________________
activation_18 (Activation)   (None, 148, 148, 16)      0         
_________________________________________________________________
batch_normalization_33 (Batc (None, 148, 148, 16)      48        
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 72, 72, 32)        4608      
_________________________________________________________________
activation_19 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
batch_normalization_34 (Batc (None, 72, 72, 32)      

In [383]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall
opt = Adam(lr = 1e-4)
model.compile(loss='binary_crossentropy', optimizer= opt,
                                                metrics=['acc',
                                                        Precision(name='precision'),
                                                        Recall(name = 'recall')])

In [384]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   target_size = (150,150),
                                                   batch_size = 20,
                                                   class_mode = 'binary')

val_generator = validation_datagen.flow_from_directory(test_dir, 
                                                   target_size = (150,150),
                                                   batch_size = 20,
                                                   class_mode = 'binary')

Found 2600 images belonging to 2 classes.
Found 460 images belonging to 2 classes.


In [387]:
from tf_explain.callbacks.grad_cam import GradCAMCallback
from tensorflow.keras import callbacks

model_checkpoint = callbacks.ModelCheckpoint('xray_model.h5',
                                                             save_best_only = True)

early_stopping = callbacks.EarlyStopping(patience = 10,
                                                         restore_best_weights =True
                                                         )
my_callbacks = [ 
                GradCAMCallback(validation_data= val_generator,
                layer_name="dense_17", 
                class_index=0,
                output_dir='output_dir0',                                
    ),
                callbacks.TensorBoard(
                log_dir='my_log_dir',
                histogram_freq=1,
                embeddings_freq=1)
]

In [388]:
TRAINING_IMAGES_COUNT = 2600
VALIDATION_IMAGES_COUNT = 460
BS = 32
EPOCHS = 50
history = model.fit_generator(train_generator,
                             steps_per_epoch = TRAINING_IMAGES_COUNT // BS,
                             epochs = EPOCHS,
                             validation_data = val_generator ,
                              validation_steps = VALIDATION_IMAGES_COUNT // BS,
                             callbacks = [early_stopping, model_checkpoint])

Epoch 1/50
81/81 [==============================] - 77s 950ms/step - loss: 0.0209 - acc: 0.9926 - precision: 0.9938 - recall: 0.9913 - val_loss: 4.9361 - val_acc: 0.7500 - val_precision: 0.6667 - val_recall: 0.9855
Epoch 2/50
81/81 [==============================] - 80s 983ms/step - loss: 8.3545e-04 - acc: 0.9994 - precision: 1.0000 - recall: 0.9988 - val_loss: 4.5902 - val_acc: 0.7643 - val_precision: 0.6919 - val_recall: 0.9932
Epoch 3/50
81/81 [==============================] - 79s 975ms/step - loss: 7.6683e-04 - acc: 0.9994 - precision: 1.0000 - recall: 0.9987 - val_loss: 6.8427 - val_acc: 0.6750 - val_precision: 0.6062 - val_recall: 0.9856
Epoch 4/50
81/81 [==============================] - 76s 938ms/step - loss: 0.0035 - acc: 0.9988 - precision: 0.9988 - recall: 0.9988 - val_loss: 5.7309 - val_acc: 0.7321 - val_precision: 0.6404 - val_recall: 0.9848
Epoch 5/50
81/81 [==============================] - 75s 921ms/step - loss: 0.0075 - acc: 0.9981 - precision: 0.9976 - recall: 0.9988

In [217]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(val_dir, 
                                                   target_size = (150,150),
                                                   batch_size = 20,
                                                   class_mode = 'binary')

Found 16 images belonging to 2 classes.


In [222]:
predIs = model.evaluate(test_generator, steps=25)

 1/25 [>.............................] - 0s 14ms/step - loss: 0.6165 - acc: 0.8125 - precision: 1.0000 - recall): 0.6250
